In [18]:
import os
import pandas as pd

from data_cleaning.dest_normalization import *

In [19]:

file_path = '../../data/type_consistent_data.parquet'

if not os.path.exists(file_path):
    print(f"File not found: {file_path}")

df = pd.read_parquet(file_path)
df.head()

df.dtypes

TripID                          int64
StartLatitude                 float64
StartLongitude                float64
StartTime         datetime64[ns, UTC]
EndLatitude                   float64
EndLongitude                  float64
EndTime           datetime64[ns, UTC]
StartPort                    category
EndPort                      category
time              datetime64[ns, UTC]
shiptype                        int64
Length                          int64
Breadth                         int64
Draught                       float64
Latitude                      float64
Longitude                     float64
SOG                           float64
COG                           float64
TH                              int64
Destination            string[python]
AisSourcen             string[python]
dtype: object

## ==================================== Remark
Destinations are being processed so only official ports are left and nothing else (can be changed by looking for NOTE comment)
## ====================================

## ==================================== Step 1
Deal with Inconsistent Records (We can have different formats representing the same thing) if they are there.
And convert columns to categorical where appropriate.
## ====================================

In [20]:
def check_unique_values(df):
    """Check unique values in each column of the DataFrame."""
    col_un = {}
    for col in df.columns:
        clean_series = df[col].dropna()
        nunique = clean_series.nunique()
        col_un[col] = nunique
    return col_un

unique_values_before = check_unique_values(df)
unique_values_before

{'TripID': 1126,
 'StartLatitude': 28,
 'StartLongitude': 34,
 'StartTime': 953,
 'EndLatitude': 29,
 'EndLongitude': 47,
 'EndTime': 943,
 'StartPort': 2,
 'EndPort': 2,
 'time': 414193,
 'shiptype': 11,
 'Length': 107,
 'Breadth': 36,
 'Draught': 238,
 'Latitude': 273,
 'Longitude': 1285,
 'SOG': 227,
 'COG': 3602,
 'TH': 361,
 'Destination': 140,
 'AisSourcen': 224}

Column: StartPort, Unique values: 2 - seems correct
Column: EndPort, Unique values: 2 - seems correct

Column: Destination, Unique values: 140 - weird
Column: AisSourcen, Unique values: 224 - should check if it is correct

In [21]:
# Case normalization
text_columns = df.select_dtypes(include=['string']).columns
for col in text_columns:
    df[col] = df[col].str.upper()  # Ensure string type and uppercase

unique_values_after = check_unique_values(df)

changed_columns = list(filter(lambda col: unique_values_after[col] != unique_values_before[col], df.columns))
df[changed_columns].dropna().nunique() # Check how many unique values are there after case normalization

Destination    139
dtype: int64

Case normalization has changed the unique values in the following columns: Destination
It seems there are still inconsistencies in the Destination column, so we will need to clean it further.

In [22]:
df['Destination'].unique() #See some examples of the Destination column

<StringArray>
[            'HAMBURG',               'DEHAM',               'DEBRE',
               'DEBRV',             'ELBE.RC',          'HAMBURG.:)',
 'BREMERHAVEN.VIA.NOK',              'DE.HAM',    'DE.HAMBIVER.ELBE',
             'ELBE.PS',
 ...
            'HALMSTAD',               'GDYNA',         'GDANSK...AS',
        'BRUNSBUETTEL',         'KALININGRAD',            'SZCZECIN',
               'SEHAD',       'GDANSK.VIANOK',           'GDYNIA.PL',
              'GDANKS']
Length: 139, dtype: string

In [23]:
df['Destination'] = df['Destination'].where(
        df['Destination'].str.contains(r'[A-Za-z]', na=False),
        "NAN"
    ) # Atle ast one alphabetic

df['Destination'] = df['Destination'].apply(
    lambda x: "NAN" if re.match(r'^[A-Z]{2}$', str(x)) else x
) #only country code

def find_values_with_special_chars(df):
    """Find values with special characters in the 'Destination' column."""
    return [
        value for value in df['Destination'].unique()
        if re.search(r'[^A-Za-z0-9]', str(value))
    ]


In [24]:
dest_before = find_values_with_special_chars(df)
print(len(dest_before), "unique values before cleaning with special characters")
dest_before

99 unique values before cleaning with special characters


['ELBE.RC',
 'HAMBURG.:)',
 'BREMERHAVEN.VIA.NOK',
 'DE.HAM',
 'DE.HAMBIVER.ELBE',
 'ELBE.PS',
 'HHLO.PS',
 'HAMBURG.DE',
 'HAMBURG/.AIRBUS',
 'BLEXEN.ROAD',
 'BRV.PS',
 'HH.FINKENWERDER',
 'DEBRV.>.DEHAM',
 'DEHAM.ELBE.PLT',
 'DEHAM.ELBE',
 'DEHAM.CTT',
 'SEAHU.>.DEBRV',
 'DEBRT.>.DEHAM',
 'DE.BRV>DEHAM',
 'DE.BRV>DE.HAM',
 'HAMBURG..DE',
 'DEBRV>DEHAM',
 'DEHAM.EGH',
 'DEHAM.CTA',
 'DEHAM.EG',
 'DE.HAM.............',
 'DE.WVN.>.DE.HAM',
 'DEBHV.NOK',
 'DEBRV.EGH',
 'DEBRV---->DEHAM',
 'HAMBURG/EUR',
 'DEBRV.>.DEIM',
 'HH.PS',
 'HH.CTB',
 'HAM.PS',
 'HAMBURG....',
 'HAMBURG???',
 'GDYNIA.VIA.NOK',
 'GDYNIA.VIA.NOK.:)',
 'GDYNIAVIA)NOK',
 'KLJ.VIA.NOK',
 'PLGDN.VIA.NOK',
 'GDYNIA.VIA',
 'GDYNIA.VIAE',
 'GDYNIA.VIBIA.NOK',
 'GDYNIA.VI',
 "'GDYNIA.VIK?0\\\\BPO?_'",
 'GDYNIA.VIE',
 'GDANSK.VIA.NOK',
 'GDYNIA...TH',
 'GDANSK.VICEL',
 'KLAIPEDA.VIA.NOK',
 'KLAIPEDA..=SWIN',
 'GDANSK....=SWIN',
 'GDANSK.VIA.KIEL.K',
 'PL.GDY',
 'PL.GDY.VIA.NOK',
 'GDYNIA...!RSBURG',
 'KLAIPEDA.VIA.NOC',
 'GD

## ==================================== Step 2
We can see that we have different formats representing the same thing, like 'HAMBURG' and 'DEHAM' ext.
The data is incredibly messy, we need to handle country codes, special characters, and different formats.

1. Some have country codes
2. Some contain starting port too
3. Some contain type of facilities (e.g., 'ELBE.RC', 'BREMERHAVEN.VIA.NOK')

start > destination
## ====================================

In [25]:
# Clean all data
df['Destination'] = df['Destination'].apply(clean_destination)
df['Destination'].unique()

array(['HAMBURG', 'DEHAM', 'DEBRE', 'DEBRV', 'ELBE.RC',
       'BREMERHAVEN.VIA.NOK', 'DE.HAM', 'DE.HAMBIVER.ELBE', 'ELBE.PS',
       'HHLO.PS', 'HAMBURG.DE', 'HAMBURG/AIRBUS', 'BREMENHAVEN',
       'FINKENWERDER', 'BLEXEN.ROAD', 'BRV.PS', 'HH.FINKENWERDER',
       'DEBRV>DEHAM', 'COPENHAGEN', 'DEHAM.ELBE.PLT', 'DEHAM.ELBE',
       'DEHAM.CTT', 'SEAHU>DEBRV', 'DEBRT>DEHAM', 'STADE', 'DE.BRV>DEHAM',
       'DE.BRV>DE.HAM', 'DEHAMCTA', 'BREMERHAVEN', 'DEHAM.EGH',
       'DEHAM.CTA', 'DEHAM.EG', 'HAMBUG', 'DEBHV', 'DE.WVN>DE.HAM',
       'DEBHV.NOK', 'NAN', 'DEBRV.EGH', 'HAMBURG/EUR', 'DEBRV>DEIM',
       'GDANSK', 'HH.PS', 'HH.CTB', 'HAM.PS', 'FINKENWERD', 'NORDENHAM',
       'GDYNIA.VIA.NOK', 'GDYNIA', 'GYDINIA', 'GDYNIAVIANOK',
       'KLJ.VIA.NOK', 'PLGDY', 'PLGDN.VIA.NOK', 'GDYNIA.VIA',
       'GDYNIA.VIAE', 'GDYNIA.VIBIA.NOK', 'GDYNIA.VI', 'GDYNIA.VIK0/BPO',
       'GDYNIA.VIE', 'GDANSK.VIA.NOK', 'GDYNIA.TH', 'GDANSK.VICEL',
       'KLAIPEDA.VIA.NOK', 'KLAIPEDA', 'KLAIPEDA.SWIN', 'G

In [26]:
# Create mask for rows containing '>'
mask = df['Destination'].str.contains('>', na=False)

# Initialize columns (if not already done)
# df['start_fr_dest'] = None
# df['cleaned_destination'] = df['Destination'].copy()

# Split and assign values safely
# df.loc[mask, 'start_fr_dest'] = df.loc[mask, 'Destination'].str.split('>').str[0] #NOTE for now we wont bother
df.loc[mask, 'Destination'] = df.loc[mask, 'Destination'].str.split('>').str[1]
df

,TripID,StartLatitude,StartLongitude,StartTime,EndLatitude,EndLongitude,EndTime,StartPort,EndPort,time,...,Length,Breadth,Draught,Latitude,Longitude,SOG,COG,TH,Destination,AisSourcen
0,39131,53.57,8.53,2016-01-24 08:06:00+00:00,53.53,9.90,2016-01-24 16:44:00+00:00,BREMERHAVEN,HAMBURG,2016-01-24 08:07:00+00:00,...,277,42,11.54,53.57,8.53,0.7,331.2,143,HAMBURG,DAIS1.81B.90B.71.71A
1,39131,53.57,8.53,2016-01-24 08:06:00+00:00,53.53,9.90,2016-01-24 16:44:00+00:00,BREMERHAVEN,HAMBURG,2016-01-24 08:10:00+00:00,...,277,42,11.54,53.57,8.53,1.6,315.3,117,HAMBURG,DAIS1.81B.90B.71.71A
2,39131,53.57,8.53,2016-01-24 08:06:00+00:00,53.53,9.90,2016-01-24 16:44:00+00:00,BREMERHAVEN,HAMBURG,2016-01-24 08:10:00+00:00,...,277,42,11.54,53.57,8.53,2.8,322.6,100,HAMBURG,DAIS1.81B.90B.71.71A
3,39131,53.57,8.53,2016-01-24 08:06:00+00:00,53.53,9.90,2016-01-24 16:44:00+00:00,BREMERHAVEN,HAMBURG,2016-01-24 08:12:00+00:00,...,277,42,11.54,53.57,8.53,2.8,286.3,74,HAMBURG,DAIS1.81B.90B.71.71A
4,39131,53.57,8.53,2016-01-24 08:06:00+00:00,53.53,9.90,2016-01-24 16:44:00+00:00,BREMERHAVEN,HAMBURG,2016-01-24 08:16:00+00:00,...,277,42,11.54,53.57,8.53,4.3,333.1,333,HAMBURG,DAIS1.81B.90B.71.71A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060703,2204049,54.36,10.14,2017-04-03 07:54:00+00:00,54.38,18.66,2017-04-04 15:28:00+00:00,KIEL,GDYNIA,2017-04-04 13:57:00+00:00,...,89,13,4.00,54.51,18.75,7.2,221.0,215,GDANSK,H7001
1060704,2204049,54.36,10.14,2017-04-03 07:54:00+00:00,54.38,18.66,2017-04-04 15:28:00+00:00,KIEL,GDYNIA,2017-04-04 13:56:00+00:00,...,89,13,4.00,54.51,18.75,7.2,221.9,215,GDANSK,H7001
1060705,2204049,54.36,10.14,2017-04-03 07:54:00+00:00,54.38,18.66,2017-04-04 15:28:00+00:00,KIEL,GDYNIA,2017-04-04 13:55:00+00:00,...,89,13,4.00,54.51,18.75,7.2,222.1,215,GDANSK,H7001
1060706,2204049,54.36,10.14,2017-04-03 07:54:00+00:00,54.38,18.66,2017-04-04 15:28:00+00:00,KIEL,GDYNIA,2017-04-04 13:54:00+00:00,...,89,13,4.00,54.51,18.76,7.2,221.2,215,GDANSK,H7001


#### **I CREATED CUSTOM FILE WITH FUNCTION WE WILL USE DOWN**

df_recombined['Destination'].unique()
Now we have somewhat cleaned Destination column, but we still have some inconsistencies. That the same ports have different names.
I didn't find quicker way to do it, rather than manually checking the names and creating a list of names that represent the same port.
We will only a bit automize this process, by using fuzzy matching to find similar names.
And extracting all ports from UpdatedPub150.csv file, which contains ports and their countries.
[link](https://msi.nga.mil/Publications/WPI)

In [27]:
# ports_cvs = '../data/UpdatedPub150.csv'
# port_df = pd.read_csv(ports_cvs)
# save_filtered_ports(port_df, country_name='Poland')
# save_filtered_ports(port_df, country_name='Germany')
# save_filtered_ports(port_df, country_name='Lithuania')
# save_filtered_ports(port_df, country_name='Sweden')

In [28]:
unique_dests = df['Destination'].unique().tolist()

# Find matches with threshold of 85
matches = find_fuzzy_matches(unique_dests, threshold=75, show_progress=True)

# Print results grouped by similarity
print_fuzzy_matches(matches, min_score=75, group_similar=True)

Processing 116/116: GDANKS......K...F.....

Fuzzy matches (score ≥ 75):

Group: BREMENHAVEN
--------------------------------------------------
  → BREMERHAVEN (score: 91)
    → BREMERHAVEN.VIA.NOK (score: 100)

Group: DE.HAM
--------------------------------------------------
  → DEHAM (score: 91)
    → DEHAM.ELBE.PLT (score: 100)
    → DEHAM.ELBE (score: 100)
    → DEHAM.CTT (score: 100)
    → DEHAM.EGH (score: 100)
    → DEHAM.CTA (score: 100)
    → DEHAM.EG (score: 100)
    → DEHAMCTA (score: 77)
  → HAMBURG.DE (score: 75)
    → HAMBURG (score: 100)
    → HAMBURG/EUR (score: 86)
    → HAMBURG/AIRBUS (score: 82)
    → HAMBUG (score: 75)

Group: DE.HAMBIVER.ELBE
--------------------------------------------------
  → DEHAM.ELBE (score: 77)
    → DEHAM (score: 100)
    → DEHAM.ELBE.PLT (score: 100)
    → DEHAM.EG (score: 78)

Group: DEBHV
--------------------------------------------------
  → DEBHV.NOK (score: 100)
    → NOK (score: 100)
  → DEBRV (score: 80)
    → DEBRV.EGH (score: 100)

From this I will manually create a list of names that represent the same port, that do not have 100 match.
As they can be incorrectly matched, and it would be better to do it manually.

https://www.marinetraffic.com/en/ais/details/ports/347?name=HALMSTAD&country=Sweden


In [29]:
# df['start_fr_dest'].unique()

In [30]:
name_german = {
    'DEHAM': ["HAMBURG", "HAMBUG", "HH", "HAM"],
    'DEBRV': ["BREMERHAVEN", "BREMENHAVEN", "BRV", "DEBHV", "BHV"],
    'DEBRE': ["BREMEN", "BRE"],
    'DEKEL': ["KIEL", "KEL"],
    'DEHAM.FINKENWERDER': ["FINKENWERDER", "FINKENWERD"],
    'DEHAM.BLEXEN': ["BLEXEN"],
    'DESTA': ["STADE", "STAD", "STA"],
    'DEBRB' : ["BRUNSBUETTEL", "BRUNSBUETT", "BRB"],
    'DEHAM.ELBE': ["ELBE"],
    'DEVTT': ["HIDDENSEE", "VTT"],
    'DEWVN': ["WILHELMSHAVEN", "WVN"],
    'country': ["DE"]
}

name_poland = {
    'PLGDN': ["GDANSK", "GDANK", "GDN"],
    'PLGDY': ["GDYNIA", "GYDNIA", "GYDINIA", "GDY", "GDYNA"],
    'SZCZECIN': ["SZCZECIN", "SZCZECIN", "SZZ"],
     'country': ["PL"]
}

name_lythuania = {
    'LTKLJ': ["KLAIPEDA", "KLJ"],
    'country': ["LT"]
}

name_sweden = {
    'SEHAD': ["HALMSTAD", "HAD"],
    'SENOK': ["NOK"],
    'SEAHU': ["AHUS", "AHU"],
    'country': ["SE"]
}

name_russia = {
        'RUKGD': ["KALININGRAD", "KALININGRAD", "KAL"],
        'country': ["RU"]
}

name_denmark = {
    'DKKOB': ["KOBENHAVN", "COPENHAGEN", "COPENHAGUE", "CPH"],
    'country': ["DK"]
}

name_finland = {
    'FIHKO': ["HANKO", "HKO"],
    'country': ["FI"]
}

name_belgium = {
    "BEANR": ["BEANR", "ANR"],
    'country': ["BE"]
}
full_dict = [name_german, name_poland, name_lythuania, name_sweden, name_russia, name_finland, name_belgium]

def replace_with_key(df, column, name_variants):
    df[column] = df[column].apply(lambda x: match_names(x, name_variants))
    return df

In [31]:
df = replace_with_key(df, 'Destination', full_dict)
# df = replace_with_key(df, 'start_fr_dest', full_dict)
df[['Destination']].reset_index().drop_duplicates(subset=['Destination'])

,index,Destination
0,0,DEHAM
2894,2894,DEBRE
5160,5160,DEBRV
12382,12382,DEHAM.ELBE.RC
13229,13229,DEBRV.VIA.NOK
...,...,...
982201,982201,RUKGD
998470,998470,SZCZECIN
1004254,1004254,SEHAD
1039679,1039679,PLGDN.VIANOK


In [32]:
mask = df['Destination'].str.contains('.', na=False)
df.loc[mask, 'Destination'] = df.loc[mask, 'Destination'].str.split('.').str[0] # NOTE Remove everything after dot

In [33]:
# df[['start_fr_dest']].reset_index().drop_duplicates(subset=['start_fr_dest'])

361854,387562,NORDEN.DEHAM
Theoretically, I can switch places for it, but I am lazy

Do we need to split it too? like debrv.via.nok?


In [34]:
save_path = '../../data/normalized_destinations.parquet'
df.to_parquet(save_path)